**Importing the required libraries**

In [202]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [203]:
# loading the csv file into pandas dataframe 
telc_df = pd.read_csv('/content/drive/MyDrive/Datasets/churn_prediction/tel_churan (1).csv')

In [204]:
# first 5 rows of the dataset
telc_df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 12,tenure_group_13 24,tenure_group_25 36,tenure_group_37 48,tenure_group_49 60,tenure_group_61 72
0,0,0,29.85,29.85,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0


In [205]:
# total number of rows and columns
telc_df.shape

(7032, 52)

In [206]:
telc_df=telc_df.drop('Unnamed: 0',axis=1)

Splitting the Feature and Target

In [207]:
x = telc_df.drop('Churn',axis = 1)
y = telc_df['Churn']

In [208]:
print(x)

      SeniorCitizen  MonthlyCharges  ...  tenure_group_49 60  tenure_group_61 72
0                 0           29.85  ...                   0                   0
1                 0           56.95  ...                   0                   0
2                 0           53.85  ...                   0                   0
3                 0           42.30  ...                   0                   0
4                 0           70.70  ...                   0                   0
...             ...             ...  ...                 ...                 ...
7027              0           84.80  ...                   0                   0
7028              0          103.20  ...                   0                   1
7029              0           29.60  ...                   0                   0
7030              1           74.40  ...                   0                   0
7031              0          105.65  ...                   0                   1

[7032 rows x 50 columns]


In [209]:
print(y)

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64


Spiltting the data into Training data and Test Data


In [210]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [256]:
print(x.shape, x_train.shape, x_test.shape)

(7032, 50) (5625, 50) (1407, 50)


Model Evaluation

In [211]:
model = DecisionTreeClassifier(criterion='gini', random_state=100, max_depth=6, min_samples_leaf=8)

In [212]:
model.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [213]:
y_pred = model.predict(x_test)

In [214]:
y_pred

array([0, 1, 0, ..., 0, 0, 0])

In [215]:
model.score(x_test,y_test)

0.7853589196872779

In [216]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1016
           1       0.65      0.50      0.56       391

    accuracy                           0.79      1407
   macro avg       0.74      0.70      0.71      1407
weighted avg       0.77      0.79      0.78      1407



As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.
Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.
Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [217]:
# confusion matrix
print(confusion_matrix(y_test, y_pred))

[[911 105]
 [197 194]]


Using SMOTEENN

In [218]:
sm = SMOTEENN()     # SMOTEENN = Over-sampling using SMOTE and cleaning using ENN. Combine over- and under-sampling using SMOTE and Edited Nearest Neighbours.
X_resampled, y_resampled = sm.fit_resample(x,y)

In [219]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [220]:
model_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [221]:
# fit and predict the model
model_smote.fit(xr_train, yr_train)
yr_predict = model_smote.predict(xr_test)


In [222]:
# score on test dataset
model_score = model_smote.score(xr_test,yr_test)

In [223]:
print(model_score)

0.9393939393939394


In [224]:
# model classification report
print(metrics.classification_report(yr_test,yr_predict, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93       548
           1       0.93      0.96      0.94       640

    accuracy                           0.94      1188
   macro avg       0.94      0.94      0.94      1188
weighted avg       0.94      0.94      0.94      1188



Now we can see quite better results, i.e. Accuracy: 95 %, and a very good recall, precision & f1 score for minority class.
Let's try with some other classifier

In [225]:
# Confusion metricx
print(metrics.confusion_matrix(yr_test,yr_predict))

[[503  45]
 [ 27 613]]


**Random Forest Classifier**

In [226]:
from sklearn.ensemble import RandomForestClassifier

In [227]:
model_rf = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=100, max_depth=6, min_samples_leaf=8)    
model_rf.fit(x_train, y_train)
y_pred_rf = model_rf.predict(x_test)

In [228]:
print(metrics.classification_report(y_test,y_pred_rf, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      1016
           1       0.70      0.41      0.52       391

    accuracy                           0.79      1407
   macro avg       0.75      0.67      0.69      1407
weighted avg       0.77      0.79      0.77      1407



In [229]:
sm = SMOTEENN()     # SMOTEENN = Over-sampling using SMOTE and cleaning using ENN. Combine over- and under-sampling using SMOTE and Edited Nearest Neighbours.
X_resampled, y_resampled = sm.fit_resample(x,y)

In [230]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [231]:
model_smote_rf=RandomForestClassifier(n_estimators=100, criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [232]:
# fit and predict the model
model_smote_rf.fit(xr_train, yr_train)
yr_predict_rf = model_smote_rf.predict(xr_test)


In [233]:
# score on test dataset
model_score = model_smote.score(xr_test,yr_test)

In [234]:
print(model_score)

0.928448275862069


In [235]:
# model classification report
print(metrics.classification_report(yr_test,yr_predict_rf, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.96      0.89      0.93       541
           1       0.91      0.97      0.94       619

    accuracy                           0.93      1160
   macro avg       0.94      0.93      0.93      1160
weighted avg       0.94      0.93      0.93      1160



In [236]:
# Confusion metricx
print(metrics.confusion_matrix(yr_test,yr_predict_rf))

[[483  58]
 [ 18 601]]


**Perfroming PCA**

In [237]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train)
xr_test_pca = pca.transform(xr_test)
explained_variance = pca.explained_variance_ratio_

In [238]:
model_pca=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [240]:
model_pca.fit(xr_train_pca, yr_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [242]:
yr_predict_pca = model_pca.predict(xr_test_pca)

In [244]:
model_score_r_pca = model_pca.score(xr_test_pca, yr_test)

In [246]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test, yr_predict_pca))

0.7655172413793103
              precision    recall  f1-score   support

           0       0.81      0.65      0.72       541
           1       0.74      0.87      0.80       619

    accuracy                           0.77      1160
   macro avg       0.77      0.76      0.76      1160
weighted avg       0.77      0.77      0.76      1160



With PCA, we couldn't see any better results, hence let's finalise the model which was created by RF Classifier, and save the model so that we can use it in a later stage :)
 

**Pickling the Model** (saving the Model)

In [247]:
import pickle

In [248]:
filename = 'model.sav'

In [249]:
pickle.dump(model_smote_rf, open(filename, 'wb'))

In [252]:
load_model = pickle.load(open(filename, 'rb'))

In [253]:
load_model

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [254]:
load_model.score(xr_test, yr_test)

0.9344827586206896